In [37]:
%pip install faiss-cpu==1.7.4 chromadb>=0.3.21

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [38]:
%%markdown
### Reading Data

### Reading Data


In [2]:
import pandas as pd
pdf = pd.read_csv(f'/home/jellyfish/llms/databricks/data/uci-news-aggregator.csv')
pdf

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027
...,...,...,...,...,...,...,...,...
422414,422933,Surgeons to remove 4-year-old's rib to rebuild...,http://www.cbs3springfield.com/story/26378648/...,WSHM-TV,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.cbs3springfield.com,1409229190251
422415,422934,Boy to have surgery on esophagus after battery...,http://www.wlwt.com/news/boy-to-have-surgery-o...,WLWT Cincinnati,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wlwt.com,1409229190508
422416,422935,Child who swallowed battery to have reconstruc...,http://www.newsnet5.com/news/local-news/child-...,NewsNet5.com,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.newsnet5.com,1409229190771
422417,422936,Phoenix boy undergoes surgery to repair throat...,http://www.wfsb.com/story/26368078/phoenix-boy...,WFSB,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wfsb.com,1409229191071


In [40]:
%%markdown
### Vector Library: FAISS

### Vector Library: FAISS


In [41]:
from sentence_transformers import InputExample
pdf_subset = pdf.head(1000)

def example_create_fn(doc1: pd.Series)->InputExample:
    """ 
        Helper function that output a sentence_transformer guid,label,text
    """
    return InputExample(texts=[doc1])

faiss_train_examples = pdf_subset.apply(lambda x: example_create_fn(x["TITLE"]),axis=1).tolist()

In [42]:
faiss_train_examples

In [43]:
%%markdown
### Vectorize text into embedding vectors


### Vectorize text into embedding vectors


In [44]:
from sentence_transformers import SentenceTransformer

In [45]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2",
    cache_folder =r'/home/jellyfish/llms/databricks/data',
)

In [46]:
faiss_title_embedding = model.encode(pdf_subset.TITLE.values.tolist())
print(len(faiss_title_embedding),len(faiss_title_embedding[0]))

1000 384


In [47]:
%%markdown
### Saving embedding vectors to FAISS index

### Saving embedding vectors to FAISS index


In [48]:
import numpy as np
import faiss


In [49]:
pdf_to_index = pdf_subset.set_index(['ID'],drop=False)
pdf_to_index

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
ID,,,,,,,,
1,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
2,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
3,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
4,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
5,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027
...,...,...,...,...,...,...,...,...
996,996,Titanfall Xbox One resolution update isn't rea...,http://www.slashgear.com/titanfall-xbox-one-re...,SlashGear,t,dw0Jnfj8vEPaLZM8TQMSCbVaYeyrM,www.slashgear.com,1394493719210
997,997,Next-Gen Starts Tomorrow With Titanfall Launch,http://www.newsledge.com/next-gen-starts-tomor...,The News Ledge,t,dw0Jnfj8vEPaLZM8TQMSCbVaYeyrM,www.newsledge.com,1394493719387
998,998,Titanfall On Xbox One Likely To Get Improved R...,http://thekoalition.com/2014/03/titanfall-on-x...,The Koalition,t,dw0Jnfj8vEPaLZM8TQMSCbVaYeyrM,thekoalition.com,1394493719614


In [50]:
id_index = np.array(pdf_to_index.ID.values).flatten().astype('int')

In [51]:
id_index

array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,
         23,   24,   25,   26,   27,   28,   29,   30,   31,   32,   33,
         34,   35,   36,   37,   38,   39,   40,   41,   42,   43,   44,
         45,   46,   47,   48,   49,   50,   51,   52,   53,   54,   55,
         56,   57,   58,   59,   60,   61,   62,   63,   64,   65,   66,
         67,   68,   69,   70,   71,   72,   73,   74,   75,   76,   77,
         78,   79,   80,   81,   82,   83,   84,   85,   86,   87,   88,
         89,   90,   91,   92,   93,   94,   95,   96,   97,   98,   99,
        100,  101,  102,  103,  104,  105,  106,  107,  108,  109,  110,
        111,  112,  113,  114,  115,  116,  117,  118,  119,  120,  121,
        122,  123,  124,  125,  126,  127,  128,  129,  130,  131,  132,
        133,  134,  135,  136,  137,  138,  139,  140,  141,  142,  143,
        144,  145,  146,  147,  148,  149,  150,  1

In [54]:
content_encoded_normalized = faiss_title_embedding.copy()
faiss.normalize_L2(content_encoded_normalized)
index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(faiss_title_embedding[0])))
index_content.add_with_ids(content_encoded_normalized,id_index)